In [ ]:
import requests
import pandas as pd

url = "https://storage.googleapis.com/asa-demo-loan-data/loan_training_data_synthetic"
r = requests.get(url)

with open("loan_data.csv", "wb") as f:
    f.write(r.content)

df = pd.read_csv("loan_data.csv")
print("✅ File downloaded and loaded!")
df.head()

In [ ]:
import pandas as pd

# Load your CSV
df = pd.read_csv("loan_data.csv")

print("✅ Data loaded successfully!")
print("Rows:", df.shape[0])
df.head()

In [1]:
# -------------------------------------------------------
# 🧠 INTEREST RATE PREDICTION MODEL — FIXED NORMALIZATION LAYER
# -------------------------------------------------------

!pip install tensorflow tf2onnx onnx scikit-learn pandas numpy --quiet

import numpy as np
if not hasattr(np, 'object'):
    np.object = object

import pandas as pd
import os
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Normalization
from sklearn.model_selection import train_test_split
import tf2onnx, onnx, types

# -------------------------------------------------------
# ✅ Load dataset
# -------------------------------------------------------
df = pd.read_csv("loan_data.csv")
print(f"✅ Dataset loaded: {df.shape}")

drop_cols = ["max_outstanding_balance", "employment_status", "region"]
df.drop(columns=[c for c in drop_cols if c in df.columns], inplace=True, errors="ignore")

# -------------------------------------------------------
# ✅ Feature engineering
# -------------------------------------------------------
if "interest_rate" not in df.columns:
    df["interest_rate"] = (
        15
        - (df["avg_credit_score"] / 100)
        + (df["total_past_due"] * 0.3)
        + (df["avg_requested_amount"] / df["avg_annual_income"]) * 5
    )
    df["interest_rate"] = np.clip(df["interest_rate"], 5, 20)
    print("ℹ️ Created synthetic 'interest_rate' column.")

feature_cols = [
    "avg_credit_score",
    "avg_annual_income",
    "avg_requested_amount",
    "avg_requested_tenor_months",
    "total_past_due"
]
X = df[feature_cols]
y = df["interest_rate"].values.reshape(-1, 1)

# -------------------------------------------------------
# ✅ Split dataset
# -------------------------------------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# -------------------------------------------------------
# ✅ Define and adapt normalization layer
# -------------------------------------------------------
norm_layer = Normalization()
norm_layer.adapt(X_train.to_numpy().astype(np.float32))  # ✅ FIXED: ensure it's NumPy array

# -------------------------------------------------------
# ✅ Build model
# -------------------------------------------------------
model = Sequential([
    norm_layer,
    Dense(64, activation="relu"),
    BatchNormalization(),
    Dropout(0.2),
    Dense(64, activation="relu"),
    BatchNormalization(),
    Dropout(0.2),
    Dense(32, activation="relu"),
    Dense(1, activation="linear")
])
model.compile(optimizer="adam", loss="mse", metrics=["mae"])
model.summary()

# -------------------------------------------------------
# ✅ Train model
# -------------------------------------------------------
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=25,
    batch_size=32,
    verbose=1
)
print("✅ Model training complete!")

# -------------------------------------------------------
# ✅ Export to ONNX (handles raw inputs!)
# -------------------------------------------------------
if not hasattr(tf2onnx, "version"):
    tf2onnx.version = types.SimpleNamespace(git_version="unknown")

input_signature = [tf.TensorSpec([None, X_train.shape[1]], tf.float32, name="input")]

@tf.function(input_signature=input_signature)
def model_inference(input):
    return model(input)

model_proto, _ = tf2onnx.convert.from_function(
    model_inference,
    input_signature=input_signature,
    opset=13
)

os.makedirs("models/interest-rate/6", exist_ok=True)
onnx.save(model_proto, "models/interest-rate/6/model.onnx")

onnx_model = onnx.load("models/interest-rate/6/model.onnx")
onnx.checker.check_model(onnx_model)
print("✅ Model exported to ONNX successfully (v6, normalization baked in!)")



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


/tmp/ipykernel_9192/3471224235.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, 'object'):
2025-10-25 15:40:25.443972: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-25 15:40:25.444730: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-25 15:40:25.516613: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-25 15:40:27.070867: I tensorflow/core/util/port.cc:153] o

✅ Dataset loaded: (1452, 16)
ℹ️ Created synthetic 'interest_rate' column.


2025-10-25 15:40:28.169609: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

...

 Trainable params: 0 (0.00 B)

 Non-trainable params: 11 (48.00 B)

Epoch 1/25
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 114.1351 - mae: 10.3579 - val_loss: 110.1245 - val_mae: 10.1737
Epoch 2/25
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 73.3642 - mae: 8.1520 - val_loss: 91.0784 - val_mae: 9.1908
Epoch 3/25
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 37.0760 - mae: 5.4717 - val_loss: 64.0995 - val_mae: 7.5789
Epoch 4/25
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 15.4183 - mae: 3.1724 - val_loss: 40.3018 - val_mae: 5.8448
Epoch 5/25
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 7.8339 - mae: 2.1656 - val_loss: 27.8291 - val_mae: 4.7430
Epoch 6/25
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 6.7172 - mae: 2.0335 - val_loss: 19.4121 - val_mae: 3.8455
Epoch 7/25
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.1808 - mae: 1.7695 - val_loss: 14.2094 - val_mae: 3.1779
Epoch 8/25
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 5.4539 - mae: 1.8340 - val_loss: 10.3795 - val_mae: 2.6666
Epoch 9/25
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - los

I0000 00:00:1761406834.209729    9192 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1761406834.209903    9192 single_machine.cc:376] Starting new session
I0000 00:00:1761406834.273084    9192 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1761406834.273315    9192 single_machine.cc:376] Starting new session
rewriter <function rewrite_constant_fold at 0x7f738dfa0a40>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.


✅ Model exported to ONNX successfully (v6, normalization baked in!)


In [2]:
# -------------------------------------------------------
# ✅ FINAL FIXED EXPORT (Handles WrappedFunction correctly)
# -------------------------------------------------------
import tensorflow as tf
import tf2onnx
import onnx
import os
import types
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

# 🩹 Patch tf2onnx version if missing
if not hasattr(tf2onnx, "version"):
    tf2onnx.version = types.SimpleNamespace(git_version="unknown")

# 1️⃣ Wrap model inference in tf.function with explicit input signature
@tf.function(input_signature=[tf.TensorSpec([None, X_train.shape[1]], tf.float32, name="input")])
def model_inference(x):
    return model(x)

# 2️⃣ Create the concrete function
concrete_func = model_inference.get_concrete_function()

# 3️⃣ Freeze the graph (embed normalization weights as constants)
frozen_func = convert_variables_to_constants_v2(concrete_func)
graph_def = frozen_func.graph.as_graph_def()
print("✅ Model graph frozen — normalization parameters baked in!")

# 4️⃣ Convert frozen graph directly to ONNX (no get_concrete_function)
model_proto, _ = tf2onnx.convert.from_graph_def(
    graph_def,
    input_names=["input:0"],
    output_names=[frozen_func.outputs[0].name],
    opset=13,  # ✅ Stable for OpenVINO & OpenShift AI
)

# 5️⃣ Save ONNX model
os.makedirs("models/interest-rate/6", exist_ok=True)
onnx_path = "models/interest-rate/6/model.onnx"
onnx.save(model_proto, onnx_path)
print(f"✅ Model exported successfully to: {onnx_path}")

# 6️⃣ Validate the ONNX model
onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)
print("✅ ONNX validation passed — ready for inference or OpenShift deployment!")


✅ Model graph frozen — normalization parameters baked in!
Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.


I0000 00:00:1761406839.994896    9192 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1761406839.995044    9192 single_machine.cc:376] Starting new session
I0000 00:00:1761406840.049413    9192 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00:1761406840.049572    9192 single_machine.cc:376] Starting new session
rewriter <function rewrite_constant_fold at 0x7f738dfa0a40>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.


✅ Model exported successfully to: models/interest-rate/6/model.onnx
✅ ONNX validation passed — ready for inference or OpenShift deployment!


In [4]:
import onnxruntime as rt
import numpy as np
import pandas as pd

onnx_model_path = "models/interest-rate/6/model.onnx"
session = rt.InferenceSession(onnx_model_path, providers=["CPUExecutionProvider"])

input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

print(f"✅ Model loaded: {onnx_model_path}")
print(f"Input: {input_name}")
print(f"Output: {output_name}")

raw_input = {
    "avg_credit_score": 750,
    "avg_annual_income": 2200000,
    "avg_requested_amount": 200000,
    "avg_requested_tenor_months": 48,
    "total_past_due": 0.00
}
X_sample = pd.DataFrame([raw_input], dtype=np.float32)
onnx_pred = session.run([output_name], {input_name: X_sample.values})[0]

print("\n🔍 Input sent:", X_sample.to_dict(orient='records')[0])
print(f"💡 Predicted interest rate: {onnx_pred[0][0]:.2f}%")


✅ Model loaded: models/interest-rate/6/model.onnx
Input: input:0
Output: Identity:0

🔍 Input sent: {'avg_credit_score': 750.0, 'avg_annual_income': 2200000.0, 'avg_requested_amount': 200000.0, 'avg_requested_tenor_months': 48.0, 'total_past_due': 0.0}
💡 Predicted interest rate: 8.77%


In [5]:
!curl -X POST -H "Content-Type: application/json" \
-d '{"inputs": [{"name": "input:0", "shape": [1, 5], "datatype": "FP32", "data": [720, 95000, 35000, 60, 0.04]}]}' \
https://interest-rate-loan-rate-model.apps.asa-demo.7mhsq.gcp.redhatworkshops.io/v2/models/interest-rate/infer


{
    "model_name": "interest-rate",
    "model_version": "5",
    "outputs": [{
            "name": "Identity:0",
            "shape": [1, 1],
            "datatype": "FP32",
            "data": [11.088425636291504]
        }]
}

In [6]:
import requests
import json

url = "https://interest-rate-loan-rate-model.apps.asa-demo.7mhsq.gcp.redhatworkshops.io/v2/models/interest-rate/infer"
payload = {
    "inputs": [{
        "name": "input:0",
        "shape": [1, 5],
        "datatype": "FP32",
        "data": [720, 950000, 15000, 60, 0.04]
    }]
}

response = requests.post(url, json=payload, verify=False)
print(json.dumps(response.json(), indent=2))


{
  "model_name": "interest-rate",
  "model_version": "5",
  "outputs": [
    {
      "name": "Identity:0",
      "shape": [
        1,
        1
      ],
      "datatype": "FP32",
      "data": [
        6.200716972351074
      ]
    }
  ]
}


/opt/app-root/lib64/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'interest-rate-loan-rate-model.apps.asa-demo.7mhsq.gcp.redhatworkshops.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [7]:
import requests

file_path = "artifact/scaler.pkl"
upload_url = "https://storage.googleapis.com/upload/storage/v1/b/asa-demo-loan-data/o?uploadType=media&name=models/scaler.pkl"

with open(file_path, "rb") as f:
    r = requests.post(upload_url, data=f, headers={"Content-Type": "application/octet-stream"})
print(r.status_code, r.text)

200 {
  "kind": "storage#object",
  "id": "asa-demo-loan-data/models/scaler.pkl/1761846588225849",
  "selfLink": "https://www.googleapis.com/storage/v1/b/asa-demo-loan-data/o/models%2Fscaler.pkl",
  "mediaLink": "https://storage.googleapis.com/download/storage/v1/b/asa-demo-loan-data/o/models%2Fscaler.pkl?generation=1761846588225849&alt=media",
  "name": "models/scaler.pkl",
  "bucket": "asa-demo-loan-data",
  "generation": "1761846588225849",
  "metageneration": "1",
  "contentType": "application/octet-stream",
  "storageClass": "STANDARD",
  "size": "765",
  "md5Hash": "70sIBvh40mFdnDZYf+l5UA==",
  "crc32c": "K46YTg==",
  "etag": "CLnyqIC+zJADEAE=",
  "timeCreated": "2025-10-30T17:49:48.265Z",
  "updated": "2025-10-30T17:49:48.265Z",
  "timeStorageClassUpdated": "2025-10-30T17:49:48.265Z",
  "timeFinalized": "2025-10-30T17:49:48.265Z"
}

